# Лабораторная работа №5 (Проведение исследований с градиентным бустингом)

# 2. Создание бейзлайна и оценка качества
**2.a** Обучить модели из `sklearn` (для классификации и регрессии) для выбранных наборов данных  
**2.b** Оценить качество моделей (для классификации и регрессии) по выбранным метрикам


In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.metrics import accuracy_score, f1_score, mean_squared_error, r2_score
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             mean_squared_error, mean_absolute_error, r2_score)
from sklearn.model_selection import GridSearchCV

In [10]:
# 1. Загрузка данных
data = pd.read_csv('student_data.csv', sep=',')
print("Shape:", data.shape)

# (Опционально) Удаляем пропуски, если есть
data.dropna(inplace=True)

# 2. Бинарная классификация: passed = 1 (G3 >= 10) / 0 иначе
data_class = data.copy()
data_class['passed'] = (data_class['G3'] >= 10).astype(int)

X_class = data_class[['G1', 'G2', 'studytime', 'failures', 'absences']]
y_class = data_class['passed']

Xc_train, Xc_test, yc_train, yc_test = train_test_split(
    X_class, y_class, test_size=0.2, random_state=42, stratify=y_class
)

# Бейзлайн (Classification)
gb_clf = GradientBoostingClassifier(random_state=42)
gb_clf.fit(Xc_train, yc_train)
yc_pred = gb_clf.predict(Xc_test)

acc = accuracy_score(yc_test, yc_pred)
prec = precision_score(yc_test, yc_pred)
rec = recall_score(yc_test, yc_pred)
f1 = f1_score(yc_test, yc_pred)

print("Бейзлайн GradientBoosting (Classification):")
print(f"Accuracy:  {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall:    {rec:.4f}")
print(f"F1-score:  {f1:.4f}")

# 3. Регрессия: предсказываем G3
X_reg = data[['G1', 'G2', 'studytime', 'failures', 'absences']]
y_reg = data['G3']

Xr_train, Xr_test, yr_train, yr_test = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

# Бейзлайн (Regression)
gb_reg = GradientBoostingRegressor(random_state=42)
gb_reg.fit(Xr_train, yr_train)
yr_pred = gb_reg.predict(Xr_test)

mse = mean_squared_error(yr_test, yr_pred)
mae = mean_absolute_error(yr_test, yr_pred)
r2 = r2_score(yr_test, yr_pred)

print("\nБейзлайн GradientBoosting (Regression):")
print(f"MSE:  {mse:.4f}")
print(f"MAE:  {mae:.4f}")
print(f"R^2:  {r2:.4f}")

Shape: (395, 33)
Бейзлайн GradientBoosting (Classification):
Accuracy:  0.8734
Precision: 0.9388
Recall:    0.8679
F1-score:  0.9020

Бейзлайн GradientBoosting (Regression):
MSE:  2.9156
MAE:  1.0626
R^2:  0.8578


**Вывод**:  
- Создан бейзлайн градиентного бустинга для классификации (метрики: accuracy, precision, recall, f1).  
- Создан бейзлайн градиентного бустинга для регрессии (метрики: MSE, MAE, R^2).  

# 3. Улучшение бейзлайна
**3.a** Сформулировать гипотезы (препроцессинг, новые признаки, подбор гиперпараметров…)  
**3.b** Проверить гипотезы  
**3.c** Сформировать улучшенный бейзлайн  
**3.d** Обучить модели (классификация и регрессия)  
**3.e** Оценить качество  
**3.f** Сравнить с пунктом 2  
**3.g** Сделать выводы

In [11]:
# 1) Гипотеза: Удалим выбросы (absences>40) и добавим признак total_G1_G2
data_improved = data.copy()
data_improved = data_improved[data_improved['absences'] <= 40]

data_improved['total_G1_G2'] = data_improved['G1'] + data_improved['G2']

# ----- КЛАССИФИКАЦИЯ -----
data_class2 = data_improved.copy()
data_class2['passed'] = (data_class2['G3'] >= 10).astype(int)
X_class2 = data_class2[['G1','G2','studytime','failures','absences','total_G1_G2']]
y_class2 = data_class2['passed']

Xc2_train, Xc2_test, yc2_train, yc2_test = train_test_split(
    X_class2, y_class2, test_size=0.2, random_state=42, stratify=y_class2
)

# Подбор гиперпараметров для классификации
param_grid_clf = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [1.0, 0.8],
}
gb_clf_cv = GridSearchCV(GradientBoostingClassifier(random_state=42),
                         param_grid_clf, cv=5, scoring='f1', n_jobs=-1)
gb_clf_cv.fit(Xc2_train, yc2_train)

best_clf = gb_clf_cv.best_estimator_
yc2_pred = best_clf.predict(Xc2_test)

acc_best = accuracy_score(yc2_test, yc2_pred)
prec_best = precision_score(yc2_test, yc2_pred)
rec_best = recall_score(yc2_test, yc2_pred)
f1_best = f1_score(yc2_test, yc2_pred)

print("Улучшенный бейзлайн (GradientBoosting - Classification):")
print("Лучшие параметры:", gb_clf_cv.best_params_)
print(f"Accuracy:  {acc_best:.4f}")
print(f"Precision: {prec_best:.4f}")
print(f"Recall:    {rec_best:.4f}")
print(f"F1-score:  {f1_best:.4f}")

# ----- РЕГРЕССИЯ -----
X_reg2 = data_improved[['G1','G2','studytime','failures','absences','total_G1_G2']]
y_reg2 = data_improved['G3']

Xr2_train, Xr2_test, yr2_train, yr2_test = train_test_split(
    X_reg2, y_reg2, test_size=0.2, random_state=42
)

param_grid_reg = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [1.0, 0.8],
}
gb_reg_cv = GridSearchCV(GradientBoostingRegressor(random_state=42),
                         param_grid_reg, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
gb_reg_cv.fit(Xr2_train, yr2_train)

best_reg = gb_reg_cv.best_estimator_
yr2_pred = best_reg.predict(Xr2_test)

mse_best = mean_squared_error(yr2_test, yr2_pred)
mae_best = mean_absolute_error(yr2_test, yr2_pred)
r2_best = r2_score(yr2_test, yr2_pred)

print("\nУлучшенный бейзлайн (GradientBoosting - Regression):")
print("Лучшие параметры:", gb_reg_cv.best_params_)
print(f"MSE:  {mse_best:.4f}")
print(f"MAE:  {mae_best:.4f}")
print(f"R^2:  {r2_best:.4f}")

Улучшенный бейзлайн (GradientBoosting - Classification):
Лучшие параметры: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 200, 'subsample': 0.8}
Accuracy:  0.9241
Precision: 0.9796
Recall:    0.9057
F1-score:  0.9412

Улучшенный бейзлайн (GradientBoosting - Regression):
Лучшие параметры: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 50, 'subsample': 0.8}
MSE:  1.7844
MAE:  0.9020
R^2:  0.8724


# 4. Имплементация алгоритма машинного обучения (Градиентный бустинг)

**4.a** Самостоятельно имплементировать алгоритмы (для классификации и регрессии)  
**4.b** Обучить имплементированные модели (для классификации и регрессии)  
**4.c** Оценить качество имплементированных моделей  
**4.d** Сравнить результаты имплементированных моделей с пунктом 2  
**4.e** Сделать выводы  
**4.f** Добавить техники из улучшенного бейзлайна (пункт 3c)  
**4.g** Обучить модели (для классификации и регрессии)  
**4.h** Оценить качество моделей  
**4.i** Сравнить результаты моделей в сравнении с пунктом 3  
**4.j** Сделать выводы

In [12]:
class SimpleDecisionStumpRegressor:
    """
    Простейшая модель дерева глубины 1:
    - один признак
    - один порог
    - предсказания = среднее слева/справа
    """
    def __init__(self):
        self.feat = None
        self.thresh = None
        self.left_value = None
        self.right_value = None

    def fit(self, X, y):
        # Очень упрощённо: берём feat=0, thresh=median
        # Вы можете расширить логику: искать лучший feat, лучший thresh
        X = np.array(X)
        y = np.array(y)
        feat = 0
        thresh = np.median(X[:, feat])
        left_idx = X[:, feat] <= thresh
        right_idx = ~left_idx

        self.feat = feat
        self.thresh = thresh
        self.left_value = y[left_idx].mean() if len(y[left_idx]) > 0 else 0
        self.right_value = y[right_idx].mean() if len(y[right_idx]) > 0 else 0
        return self

    def predict(self, X):
        X = np.array(X)
        pred = np.zeros(X.shape[0])
        left_idx = X[:, self.feat] <= self.thresh
        right_idx = ~left_idx
        pred[left_idx] = self.left_value
        pred[right_idx] = self.right_value
        return pred


class SimpleGBRegressor:
    """
    Упрощённый градиентный бустинг для регрессии:
    - Инициализация: f0 = среднее целевой переменной
    - Каждый новый базовый learner (decision stump) обучается на псевдо-остатках
    - f_m(x) = f_{m-1}(x) + lr * h_m(x)
    """
    def __init__(self, n_estimators=10, learning_rate=0.1):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.models = []
        self.f0 = 0

    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y, dtype=float)
        
        # step 0: f0 = среднее
        self.f0 = np.mean(y)
        
        # инициализируем текущие предсказания
        f_current = np.ones_like(y) * self.f0
        
        self.models = []
        for i in range(self.n_estimators):
            # псевдо-остатки
            residuals = y - f_current
            
            # обучаем stump на residuals
            stump = SimpleDecisionStumpRegressor()
            stump.fit(X, residuals)
            self.models.append(stump)
            
            # обновляем f_current
            h_m = stump.predict(X)
            f_current += self.learning_rate * h_m
        return self

    def predict(self, X):
        X = np.array(X)
        # f0 + сумма(lr * h_m(x))
        preds = np.ones(X.shape[0]) * self.f0
        for stump in self.models:
            preds += self.learning_rate * stump.predict(X)
        return preds


# ==================
# Пример использования
# ==================

# 1) КАСТОМНАЯ РЕГРЕССИЯ
cust_gb_reg = SimpleGBRegressor(n_estimators=10, learning_rate=0.1)
cust_gb_reg.fit(Xr_train, yr_train)
yr_pred_custom = cust_gb_reg.predict(Xr_test)

mse_cust = mean_squared_error(yr_test, yr_pred_custom)
r2_cust = r2_score(yr_test, yr_pred_custom)

print("[Custom GB Regressor]")
print(f"MSE: {mse_cust:.4f}")
print(f"R^2: {r2_cust:.4f}")

# 2) Для классификации: можно сделать аналог (SimpleGBClassifier) 
#   - но обычно нужно считать логистическую ошибку / экспоненциальную и т.д.
#   - здесь покажем упрощённый вариант на MSE, превращая метки 0/1 -> float.

class SimpleGBClassifier:
    """
    Упрощённо, будем считать 0/1, и 
    оптимизировать MSE между предсказанием и метками (как регрессия).
    Потом threshold=0.5
    """
    def __init__(self, n_estimators=10, learning_rate=0.1):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.models = []
        self.f0 = 0

    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y, dtype=float)
        self.f0 = np.mean(y)
        f_current = np.ones_like(y) * self.f0
        self.models = []

        for i in range(self.n_estimators):
            residuals = y - f_current
            stump = SimpleDecisionStumpRegressor()
            stump.fit(X, residuals)
            self.models.append(stump)
            f_current += self.learning_rate * stump.predict(X)
        return self

    def predict_proba(self, X):
        # f0 + сумма(lr * h_m(x)) -> sigmoid
        raw_preds = np.ones(X.shape[0]) * self.f0
        for stump in self.models:
            raw_preds += self.learning_rate * stump.predict(X)
        # применим сигмоиду
        return 1 / (1 + np.exp(-raw_preds))

    def predict(self, X, threshold=0.5):
        proba = self.predict_proba(X)
        return (proba >= threshold).astype(int)


cust_gb_clf = SimpleGBClassifier(n_estimators=10, learning_rate=0.1)
cust_gb_clf.fit(Xc_train, yc_train)
yc_pred_custom = cust_gb_clf.predict(Xc_test)

acc_cust = accuracy_score(yc_test, yc_pred_custom)
f1_cust = f1_score(yc_test, yc_pred_custom)

print("\n[Custom GB Classifier]")
print(f"Accuracy: {acc_cust:.4f}")
print(f"F1-score: {f1_cust:.4f}")


[Custom GB Regressor]
MSE: 11.4839
R^2: 0.4399

[Custom GB Classifier]
Accuracy: 0.6709
F1-score: 0.8030
